In [ ]:
"""
For this experiment  you need to have your own csv containing types of questions that users can
ask or ask the AI team to for their dataset as we did not upload the csv to this repo
"""

In [10]:
import pandas as pd
from ai_docs_filterer_for_RAG import run_rm_labeller
from pydantic_ai.models.openai import OpenAIChatModel
from pydantic_ai.providers.openai import OpenAIProvider
from openai import AsyncAzureOpenAI
import os
import asyncio
from dotenv import load_dotenv
from ccs_website_data import  fetch_all_ccs_frameworks

load_dotenv()

True

In [15]:
test_data = pd.read_csv("query_filter_questions.csv")
ccs_frameworks = fetch_all_ccs_frameworks()
pydantic_azure_client = AsyncAzureOpenAI(
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    api_key=os.getenv("AZURE_OPENAI_KEY"),
    azure_deployment=os.getenv("DEPLOYMENT_NAME"),
    api_version=os.getenv("AZURE_OPENAI_API_VERSION"),
)
pydantic_rm_labeller_model = OpenAIChatModel(
    model_name= os.getenv("DEPLOYMENT_NAME"),
    provider=OpenAIProvider(openai_client=pydantic_azure_client)
)

rm_descriptions = "\n".join([
    f"RM: {r.rm_number} | "
    f"Keywords: {r.keywords if 'keywords' in r and str(r.keywords).strip() else 'N/A'} | "
    f"Summary: {r.summary} | "
    f"Pillar: {r.pillar} ({r.category})"
    for _, r in ccs_frameworks.iloc[:20].iterrows()# get only first 20
])


Starting to fetch and clean frameworks (Status: Live,Expired)...
Fetched Page 1. Total frameworks collected: 268
Finished. Reached the last page: 1.

✅ DataFrame created successfully!


## Running experiment

In [23]:
async def process_dataframe(df):
    # 1. Create the list of coroutines
    # We add a small helper here to extract .rm_number after the await
    async def get_label(query_text):
        result = await run_rm_labeller(pydantic_rm_labeller_model, rm_descriptions, query_text)
        return result.rm_number # Extracting the specific field from Pydantic

    # 2. Build the task list
    tasks = [get_label(row['query']) for _, row in df.iterrows()]

    # 3. Run concurrently
    results = await asyncio.gather(*tasks)

    # 4. Assign results to the dataframe
    df['predicted_rm'] = results
    return df

In [24]:
result_df = await process_dataframe(test_data)

In [25]:
print(result_df["predicted_rm"])

0     RM6348
1     RM6348
2     RM6348
3     RM6348
4     RM6348
       ...  
90    RM6111
91    RM6111
92    RM6292
93    RM6111
94    RM6292
Name: predicted_rm, Length: 95, dtype: object


In [26]:
# Calculate accuracy as a decimal (e.g., 0.85)
accuracy = (result_df['Truth Set'] == result_df['predicted_rm']).mean()

print(f"Overall Accuracy: {accuracy * 100:.2f}%")

Overall Accuracy: 82.11%
